In [ ]:
def ransac_line(X, threshold, max_iter=1000):
    best_inliers = []
    best_model = None

    for _ in range(max_iter):
        sample = X[np.random.choice(X.shape[0], 2, replace=False)]
        p1, p2 = sample
        line = np.cross(np.append(p1, 1), np.append(p2, 1))  
        a, b, d = line[0], line[1], line[2]
        norm_factor = np.sqrt(a**2 + b**2)
        a, b, d = a / norm_factor, b / norm_factor, d / norm_factor
        normal = np.array([a, b])
        distances = np.abs(np.dot(X, normal) + d) / np.linalg.norm(normal)
        inliers = X[distances < threshold]
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_model = (a, b, d)

    return best_model, best_inliers


In [ ]:
def fit_circle(points):
    def calc_R(xc, yc):
        return np.sqrt((points[:, 0] - xc) ** 2 + (points[:, 1] - yc) ** 2)
    def f_2(c):
        Ri = calc_R(*c)
        return np.std(Ri)
    center_estimate = np.mean(points, axis=0)
    center_optimized = minimize(f_2, center_estimate).x
    radii = calc_R(*center_optimized)
    return center_optimized[0], center_optimized[1], np.mean(radii)

def find_inliers(points, xc, yc, radius, threshold):
    distances = np.sqrt((points[:, 0] - xc) ** 2 + (points[:, 1] - yc) ** 2)
    inliers = points[np.abs(distances - radius) < threshold]
    return inliers